## Preprocesamiento

In [1]:
import os
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
import mlflow
import mlflow.tensorflow

In [2]:

# Directorio actual
directorio_actual = os.getcwd()
print(f'Directorio actual: {directorio_actual}')

# Obtener la ruta del directorio 'val'
train_dir = os.path.join(directorio_actual, 'train')
test_dir = os.path.join(directorio_actual, 'test')
val_dir = os.path.join(directorio_actual, 'val')




Directorio actual: C:\Users\Ricardo\Desktop\BrainTumors_CNN\dataset_final


In [3]:
batch_size = 32
target_size = (224, 224)

# Generador de datos para dataset de entrenamiento

train_datagen = ImageDataGenerator(rescale=1./255.)

train_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size)


# Generador de datos para dataset de validación
val_datagen = ImageDataGenerator(rescale=1./255.)

val_dataset = val_datagen.flow_from_directory(
    val_dir,
    target_size=target_size,
    batch_size=batch_size)

# Generador de datos para dataset de prueba
test_datagen = ImageDataGenerator(rescale=1./255.)
test_dataset = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=batch_size)


Found 2475 images belonging to 4 classes.
Found 309 images belonging to 4 classes.
Found 312 images belonging to 4 classes.


## Entrenamiento con MLflow

In [ ]:
# base_model = tf.keras.applications.VGG16(input_shape=(224,224,3),include_top=False,weights='imagenet')
# base_model.trainable = False
# model=keras.models.Sequential()
# model.add(base_model)
# model.add(keras.layers.Flatten()) 
# model.add(keras.layers.Dropout(.5))
# model.add(keras.layers.Dense(4,activation=tf.nn.softmax))
# model.summary()

In [4]:
import dagshub
import mlflow

dagshub.init("BrainTumors_CNN", "RicardoHdz12", mlflow=True)
mlflow.start_run()

Repository initialized!

<ActiveRun: >

In [6]:
import mlflow
import mlflow.tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dropout, Dense

# Definir la arquitectura del modelo
base_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(4, activation=tf.nn.softmax))

# Resumen del modelo
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dropout_1 (Dropout)         (None, 25088)             0         
                                                                 
 dense_1 (Dense)             (None, 4)                 100356    
                                                                 
Total params: 14815044 (56.51 MB)
Trainable params: 100356 (392.02 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [9]:

# Crear un experimento de MLflow
experiment_name = "tumores"
mlflow.set_experiment(experiment_name)

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Iniciar un experimento de MLflow
with mlflow.start_run(run_name="Neural Net"):

    # Registrar parámetros
    mlflow.log_param("epochs", 8)

    # Entrenar el modelo
    hist = model.fit(train_dataset, validation_data=val_dataset, epochs=8)

    # Registrar métricas
    mlflow.log_metric("final_accuracy", hist.history['accuracy'][-1])
    mlflow.log_metric("final_val_accuracy", hist.history['val_accuracy'][-1])

    # Guardar el modelo en formato TensorFlow
    mlflow.tensorflow.save_model(model, "model")


mlflow.end_run()


Epoch 1/8
78/78 [==============================] - 250s 3s/step - loss: 1.0914 - accuracy: 0.6117 - val_loss: 0.4701 - val_accuracy: 0.8155
Epoch 2/8
78/78 [==============================] - 248s 3s/step - loss: 0.4766 - accuracy: 0.8141 - val_loss: 0.5120 - val_accuracy: 0.7864
Epoch 3/8
78/78 [==============================] - 248s 3s/step - loss: 0.3313 - accuracy: 0.8784 - val_loss: 0.3540 - val_accuracy: 0.8738
Epoch 4/8
78/78 [==============================] - 248s 3s/step - loss: 0.2851 - accuracy: 0.8986 - val_loss: 0.4625 - val_accuracy: 0.8026
Epoch 5/8
78/78 [==============================] - 248s 3s/step - loss: 0.2484 - accuracy: 0.9059 - val_loss: 0.3238 - val_accuracy: 0.8932
Epoch 6/8
78/78 [==============================] - 248s 3s/step - loss: 0.1808 - accuracy: 0.9378 - val_loss: 0.3421 - val_accuracy: 0.8867
Epoch 7/8
78/78 [==============================] - 249s 3s/step - loss: 0.1538 - accuracy: 0.9471 - val_loss: 0.2747 - val_accuracy: 0.9126
Epoch 8/8
78/78 [===

2023/11/13 01:50:31 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\Ricardo\Desktop\BrainTumors_CNN\dataset_final\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\Ricardo\Desktop\BrainTumors_CNN\dataset_final\model\data\model\assets
C:\Users\Ricardo\Desktop\BrainTumors_CNN\venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
